https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_edge_manager/sagemaker_edge_example/sagemaker_edge_example.ipynb  
を一部改変して、コメントを日本語化したものです

# SageMaker EdgeManager Sample

## 概要

* Amazon SageMaker EdgeManager の機能を一通り使用するハンズオンです。
* [開発者ドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/edge.html)の流れを実行できるように[ノートブック形式にしたもの](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_edge_manager/sagemaker_edge_example/sagemaker_edge_example.ipynb)で処理を一部改変し、日本語の説明を追加したものです。

## 前提
* SageMaker Notebook インスタンスで実行
* Administrator Access がSageMakerのロールにアタッチされている

## 構成

## 手順
0. [前準備](#前準備)
  1. [このノートブックを実行するにあたっての Python モジュール読み込み](#このノートブックを実行するにあたっての Python モジュール読み込み)
  2. [既定値の取得](#既定値の取得)
  3. [定数の設定](#定数の設定)
  4. [推論用サンプル画像のS3アップロード](#推論用サンプル画像のS3アップロード)
  5. [使用するAWSのクライアントインスタンスを生成](https://my-first-notebook-oarf.notebook.ap-northeast-1.sagemaker.aws/notebooks/sagemaker-edgemanager-sample/SageMakerEdgeManagerSample.ipynb#%E4%BD%BF%E7%94%A8%E3%81%99%E3%82%8BAWS%E3%81%AE%E3%82%AF%E3%83%A9%E3%82%A4%E3%82%A2%E3%83%B3%E3%83%88%E3%82%A4%E3%83%B3%E3%82%B9%E3%82%BF%E3%83%B3%E3%82%B9%E3%82%92%E7%94%9F%E6%88%90)
1. [エッジデバイスを模したEC2を作成](#エッジデバイスを模したEC2を作成  )  
  ※本来のエッジデバイスであれば不要な作業で、エッジ側で別途OSインストールなどをしておいてください  
  ※アーティファクトをダウンロードするために、S3 の権限と、EC2 で ssh で入って処理をおこなうために、SSM を扱うためのロールとポリシーを設定します
  1. [すでにあった場合のためのポリシー、インスタンスプロファイル、ロールの事前削除](#すでにあった場合のためのポリシー、インスタンスプロファイル、ロールの事前削除)
  2. [EC2のロールにアタッチするインラインポリシーの作成](#EC2のロールにアタッチするインラインポリシーの作成)
  3. [EC2のロールを作成](EC2のロールを作成)
  3. [インスタンスプロファイルを作成してEC2のロールに追加](#インスタンスプロファイルを作成してEC2のロールに追加)
  4. [EC2のロールに各種ポリシーをアタッチ](#EC2のロールに各種ポリシーをアタッチ)
  5. [EC2のキーペアを作成](#EC2のキーペアを作成)
  6. [EC2インスタンスを起動](#EC2インスタンスを起動)
2. [エッジで動かすKerasモデルの準備](#エッジで動かすKerasモデルの準備)
  1. [MobilenetV2(pre-trained)のモデルをS3に保存](#MobilenetV2(pre-trained)のモデルをS3に保存)
  2. [MobinenetV2をNeoでコンパイル](#MobinenetV2をNeoでコンパイル)
  3. [EdgeManagerでモデルをパッケージ化](#EdgeManagerでモデルをパッケージ化)
3. [IoTの設定](#IoTの設定)
  1. [ThingとTypeを作成](#ThingとTypeを作成)
  2. [Thingに割り当てるロールを作成](#Thingに割り当てるロールを作成)
4. [SageMakerEdgeManagerの設定](#SageMakerEdgeManagerの設定)
  1. [デバイスフリートの作成](#デバイスフリートの作成)
  2. [デバイスの登録](#デバイスの登録)
5. [X.509クライアント証明書の作成と設定](#X.509クライアント証明書の作成と設定)
  1. [証明書の作成](#証明書の作成)
  2. [証明書とロールエイリアスを紐付けるポリシーを証明書にアタッチ](#証明書とロールエイリアスを紐付けるポリシーを証明書にアタッチ)
  3. [IoTエンドポイントを取得](#IoTエンドポイントを取得)
  4. [RootCAの取得](#RootCAの取得)
  5. [IoTエンドポイントの動作確認](#IoTエンドポイントの動作確認)
  6. [証明書たちをS3にアップロードする](#証明書たちをS3にアップロードする)
6. [エッジの準備と推論](#エッジの準備と推論)
  1. [Agentをエッジにダウンロード](#Agentをエッジにダウンロード)
  2. [各種ファイルをエッジ側でダウンロード](#各種ファイルをエッジ側でダウンロード)
  3. [Agentのconfigファイルの配置とAgentのdaemonを起動](#Agentのconfigファイルの配置とAgentのdaemonを起動)
  4. [モデルをAgentにロード](#モデルをAgentにロード)
  5. [Agentを利用して推論](#Agentを利用して推論)
  6. [データのキャプチャ](#データのキャプチャ)
  7. [モデルをAgentからアンロード](#モデルをAgentからアンロード)
7. [おかたづけ](#おかたづけ)

## 前準備

### このノートブックを実行するにあたっての Python モジュール読み込み

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3, botocore
import json, uuid
from time import sleep
import tarfile
from time import time
import tensorflow as tf

### 既定値の取得
使用している API はこちら
* [get_execution_role](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html?highlight=get_execution_role#sagemaker.session.get_execution_role)
* [Session](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html?highlight=get_execution_role#sagemaker.session.Session)

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()
print(f'role: {role}')
print(f'region: {region}')

### 定数の設定

In [ ]:
# 各種フォルダ・URI設定 
folder = 'DEMO-Sagemaker-Edge'
compilation_output_sub_folder =  f'{folder}/compilation-output'
iot_folder = f'{folder}/iot'
s3_compilation_output_location = f's3://{bucket}/{compilation_output_sub_folder}'

print(f'bucket: {bucket}')
print(f'folder: {folder}')
print(f'compilation_output_sub_folder: {compilation_output_sub_folder}')
print(f'iot_folder: {iot_folder}')
print(f's3_compilation_output_location: {s3_compilation_output_location}')

### 推論用サンプル画像のS3アップロード

In [ ]:
keras_img_path = sess.upload_data('keras.bmp', bucket, iot_folder)
print(f'keras_img_path: {keras_img_path}')

### 使用するAWSのクライアントインスタンスを生成

In [ ]:
# boto3 client 作成
ec2_client = boto3.client('ec2', region_name=region)
iam_client = boto3.client('iam')
sagemaker_client = boto3.client('sagemaker')
iot_client = boto3.client('iot', region_name=region)
s3_client = boto3.client('s3')
ssm_client = boto3.client('ssm', region_name=region)

## エッジデバイスを模したEC2を作成  
* エッジデバイスが手元にある人は珍しいので、EC2インスタンスを仮想エッジデバイスとして使用する
* エッジデバイスが手元にある方はこの章を飛ばして構いません
  ただし、EC2ロールを利用して、S3からダウンロードを後で行うので、ダウンロード方法は別途検討の必要があります(CloudFront + S3 でホスティングの上 wget するなど)
* EC2 を SSH で操作するために Systems Manager の Session Manager を利用するため、そのポリシーも合わせてアタッチする

### すでにあった場合のためのポリシー、インスタンスプロファイル、ロールの事前削除

In [ ]:
# 定数定義
ec2_role_name = 'edge-equivalent-ec2-role'
ec2_inline_policy_name = 'EdgeManagrSample-GetS3-FromEc2'

ec2_role_attach_policy_arn_list = [
    'arn:aws:iam::aws:policy/service-role/AmazonEC2RoleforSSM',
    'arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore',
    'arn:aws:iam::aws:policy/AmazonSSMDirectoryServiceAccess',
    'arn:aws:iam::aws:policy/CloudWatchAgentServerPolicy'
]

In [ ]:
# 既存ポリシーのデタッチ、ポリシーバージョンの削除、ポリシーの削除
# 存在しない場合はexceptでスキップされる

# インラインポリシーの削除
policies = iam_client.list_policies()
for policy in policies['Policies']:
    policy_arn = policy['Arn'] if policy['PolicyName'] in ec2_inline_policy_name else None
    if policy_arn:
        # Policy をデタッチ
        try:
            response = iam_client.detach_role_policy(
                RoleName=ec2_role_name,
                PolicyArn=policy_arn
            )
            print(f'detach {ec2_role_name} {policy_arn}')
            print(respoonse)
        except:
            print('did not detach role policy')
        # Policy バージョンの削除
        try:
            for policy_version in iam_client.list_policy_versions(PolicyArn=policy_arn)['Versions']:
                response = iam.delete_policy_version(PolicyArn=policy_arn,VersionId=policy_version['VersionId'])
                print(f'delete policy version {policy_arn} {policy_version["VersionId"]}')
                print(response)
        except:
            print('did not delete policy version')
        # Policy を削除
        try:
            response = iam_client.delete_policy(PolicyArn=policy_arn)
            print(f'delete policy {policy_arn}')
            print(response)
        except:
            print('did not delete policy')

# その他ポリシーの削除
for policy_arn in ec2_role_attach_policy_arn_list:                      
    try:
        response = iam_client.detach_role_policy(RoleName=ec2_role_name,PolicyArn=policy_arn)
        print(f'detach ec2 role {policy_arn}')
        print(json.dumps(response, indent=2))
    except:
        print(f'could not detach {policy_arn}')
                      


# インスタンスプロファイルを削除
try:
    response = iam_client.remove_role_from_instance_profile(
        InstanceProfileName=ec2_role_name,
        RoleName=ec2_role_name
    )
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to remove role from instance profile')

# ロールの削除
try:
    response = iam_client.delete_role(RoleName = ec2_role_name)
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete role')
try:
    response = iam_client.delete_instance_profile(InstanceProfileName=ec2_role_name)
    print('delete instance profile')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete instance profile')

### EC2のロールにアタッチするインラインポリシーの作成

In [ ]:
# ポリシーの作成
s3_get_from_ec2_policy_doc = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Action': [
                's3:GetObject',
                's3:PutObject',
                's3:PutObjectAcl',
            ],
            'Resource': [
                f'arn:aws:s3:::{bucket}/*',
                'arn:aws:s3:::sagemaker-edge-release-store-us-west-2-linux-armv8',
                'arn:aws:s3:::sagemaker-edge-release-store-us-west-2-linux-x64',
                'arn:aws:s3:::sagemaker-edge-release-store-us-west-2-windows-x64',
                'arn:aws:s3:::sagemaker-edge-release-store-us-west-2-windows-x86',
                f'arn:aws:s3:::aws-ssm-{region}/*',
                f'arn:aws:s3:::aws-windows-downloads-{region}/*',
                f'arn:aws:s3:::amazon-ssm-{region}/*',
                f'arn:aws:s3:::amazon-ssm-packages-{region}/*',
                f'arn:aws:s3:::{region}-birdwatcher-prod/*',
                f'arn:aws:s3:::aws-ssm-distributor-file-{region}/*',
                f'arn:aws:s3:::aws-ssm-document-attachments-{region}/*',
                f'arn:aws:s3:::patch-baseline-snapshot-{region}/*'
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "ssmmessages:CreateControlChannel",
                "ssmmessages:CreateDataChannel",
                "ssmmessages:OpenControlChannel",
                "ssmmessages:OpenDataChannel"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetEncryptionConfiguration"
            ],
            "Resource": "*"
        }
    ]
}
response = iam_client.create_policy(
    PolicyName=ec2_inline_policy_name,
    PolicyDocument=json.dumps(s3_get_from_ec2_policy_doc),
    Description='s3 get from ec2 policy',
)
s3_get_from_ec2_policy_arn = response['Policy']['Arn']
ec2_inline_policy_name = response['Policy']['PolicyName']
print(f'作成した policy の\narn は "{s3_get_from_ec2_policy_arn}"')
print(f'名前は "{ec2_inline_policy_name}"\nです')

### EC2のロールを作成

In [ ]:
# ロールの作成
assume_role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service": "ec2.amazonaws.com"},"Action": "sts:AssumeRole"}]
}
response = iam_client.create_role(
    RoleName = ec2_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
    Description='edge equivalent ec2 role',
    MaxSessionDuration=3600*12 # 12時間
)
print(response)

### インスタンスプロファイルを作成してEC2のロールに追加

In [ ]:
response = iam_client.create_instance_profile(InstanceProfileName=ec2_role_name)
print(response)

response = iam_client.add_role_to_instance_profile(
    RoleName=ec2_role_name,
    InstanceProfileName=ec2_role_name
)
print(response)

### EC2のロールに各種ポリシーをアタッチ

In [ ]:
# inline policy
response = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn=s3_get_from_ec2_policy_arn
)
print(json.dumps(response, indent=2))

# managed policy
for policy_arn in ec2_role_attach_policy_arn_list:
    response = iam_client.attach_role_policy(
        RoleName=ec2_role_name,
        PolicyArn=policy_arn
    )
    print(f'attach policy_arn')
    print(json.dumps(response, indent=2))

### EC2のキーペアを作成

In [ ]:
# 既存のキーペアがあったら削除
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x : x['KeyName'], key_pairs['KeyPairs']))

key_name = 'edge-equivalent-ec2-key-pair'

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(KeyName=key_name,)

# キーペアを作成
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

key_pair = str(ec2_key_pair['KeyMaterial'])
with open(f'./{key_name}.pem','wt') as f:
    f.write(key_pair)

### EC2インスタンスを起動

In [ ]:
ami_map = {
    'us-east-1': 'ami-063585f0e06d22308',
    'us-east-2': 'ami-01bd6a1621a6968d7',
    'us-west-2': 'ami-0bc87a16c757a7f07',
    'eu-central-1': 'ami-01227276a4e5a4a31',
    'ap-northeast-1': 'ami-03b8cfea5460e4881',
    'eu-west-1': 'ami-006ff58f5247c50eb'
}

In [ ]:
user_data = """#!/bin/bash
export work_dir=/home/ubuntu/
export log_file=/home/ubuntu/userdata.log
cd $work_dir
sudo apt update -y >> $log_file
sudo apt upgrade -y >> $log_file
sudo apt install python3-pip unzip awscli zip -y >> $log_file
pip3 install awscli >> $log_file
wget https://d190oj76e915as.cloudfront.net/PVREOnboarding.zip >> $log_file
unzip PVREOnboarding.zip -d PVREOnboarding >> $log_file
cd PVREOnboarding
echo ap-northeast-1 | bash PVRE-SSM-onboarding.sh >> $log_file
"""
print(user_data)

In [ ]:
# instance profile が有効化されるまで念の為5秒のsleepを入れておく。たまにそれでもエラーで落ちるが、しばらくして再実行すると動く。
sleep(10)

# EC2 の立ち上げ
ec2_instance = ec2_client.run_instances(
    ImageId=ami_map[region],
    MinCount=1,
    MaxCount=1,
    InstanceType='c5.large',
    KeyName=key_name,
    IamInstanceProfile={'Name': ec2_role_name},
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'edge-equivalent-ec2',
                },
            ],
        },
    ],
    UserData = user_data
)
sleep(10)
instance_id = ec2_instance['Instances'][0]['InstanceId']
print(f'instance_id : {instance_id}')

## エッジで動かすKerasモデルの準備

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_compilation_job

### MobilenetV2(pre-trained)のモデルをS3に保存
[MobileNetV2](https://keras.io/ja/applications/#mobilenetv2)を利用

In [ ]:
# モデルの呼び出し
model = tf.keras.applications.MobileNetV2()
# モデルのローカル保存
model.save('mobilenet_v2.h5')
# tar.gz で固める
with tarfile.open('mobilenet_v2.tar.gz', mode='w:gz') as archive:
    archive.add('mobilenet_v2.h5')
# S3 にアップロード
keras_model_path = sess.upload_data('mobilenet_v2.tar.gz', bucket, folder)
print(f'keras_model_path: {keras_model_path}')

### MobinenetV2をNeoでコンパイル
下記 API を利用。
* [create_compiliation_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_compilation_job)(コンパイルジョブを作成)
* [describe_compilation_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_compilation_job)(コンパイルジョブの状態を取得)

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = 'keras'
target_device = 'ml_c5'

In [ ]:
keras_compilation_job_name = 'Sagemaker-Edge-'+ str(time()).split('.')[0]
print(f'Compilation job for {keras_compilation_job_name} started')

response = sagemaker_client.create_compilation_job(
        CompilationJobName=keras_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': keras_model_path,
            'DataInputConfig': keras_model_data_shape,
            'Framework': keras_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# 30秒周期でポーリング
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=keras_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    sleep(30)
print('Done!')

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0"
keras_model_package = '{}-{}.tar.gz'.format(keras_packaged_model_name, keras_model_version)
print(f'keras_model_package: {keras_model_package}')

### EdgeManagerでモデルをパッケージ化
下記 API を使用
* [create_edge_packaging_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_edge_packaging_job)
* [describe_edge_packaging_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_edge_packaging_job)

In [ ]:
keras_packaging_job_name=keras_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=keras_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    sleep(30)
print('Done!')

In [ ]:
keras_model_data = job_status["ModelArtifact"]
print(f'keras_model_data: {keras_model_data}')

## IoTの設定

### ThingとTypeを作成
* すでにあった場合のThing,Typeなどを最初に削除する

In [ ]:
# thing の名前を定義
iot_thing_name = 'sagemaker-edge-thing-demo'
iot_thing_type = 'SagemakerEdgeDemo'

# thing の削除
try:
    iot_client.delete_thing(thingName=iot_thing_name)
    print('deleted thing')
except:
    print('nothing to delete thing')
# type の削除
try:
    iot_client.deprecate_thing_type(thingTypeName=iot_thing_type)
    print('deprecated thing type')
    sleep(60*5 + 10) # deprecate から5分立たないと delete できない。10 は buffer
except:
    print('nothing to deprecate thing type')
try:
    iot_client.delete_thing_type(thingTypeName=iot_thing_type)
    print('deleted thing type')
except:
    print('nothing to delete thing type')


response = iot_client.create_thing_type(thingTypeName=iot_thing_type)
print('created thing type')
print(response)

response = iot_client.create_thing(
    thingName=iot_thing_name,
    thingTypeName=iot_thing_type
)
print('created thing')
print(response)

### Thingに割り当てるロールの作成とポリシーのアタッチ

In [ ]:
# ロールの事前削除（存在する場合のみ）
sagemaker_edgemaneger_thing_role_name = 'sagemaker-edgemanager-thing-role'
thing_role_policy_arn_list = [
    'arn:aws:iam::aws:policy/service-role/AmazonSageMakerEdgeDeviceFleetPolicy',
    'arn:aws:iam::aws:policy/service-role/AWSIoTRuleActions',
    'arn:aws:iam::aws:policy/service-role/AWSIoTLogging',
    'arn:aws:iam::aws:policy/service-role/AWSIoTThingsRegistration'
]

for policy_arn in thing_role_policy_arn_list:    
    try:
        response = iam_client.detach_role_policy(RoleName=sagemaker_edgemaneger_thing_role_name,PolicyArn=policy_arn)
        print(f'detached {policy_arn}')
        print(json.dumps(response, indent=2))
    except:
        print(f'not attach {policy_arn}')


try:
    response = iam_client.delete_role(RoleName = sagemaker_edgemaneger_thing_role_name)
    print('delete sagemaker edgemanager thing role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete role')


assume_role_policy_document = {
    "Version": "2012-10-17","Statement": [
        {"Effect": "Allow","Principal": {"Service": "credentials.iot.amazonaws.com"},"Action": "sts:AssumeRole"},
        {"Effect": "Allow","Principal": {"Service": "sagemaker.amazonaws.com"},"Action": "sts:AssumeRole"}
    ]
}
print('ロールを作成')
response = iam_client.create_role(
    RoleName = sagemaker_edgemaneger_thing_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
    Description='edge equivalent ec2 role',
    MaxSessionDuration=3600*12 # 12時間
)
sagemaker_edgemaneger_thing_role_arn = response['Role']['Arn']
print(response)

print('ポリシーをアタッチ')
for policy_arn in thing_role_policy_arn_list:
    response = iam_client.attach_role_policy(
        RoleName=sagemaker_edgemaneger_thing_role_name,
        PolicyArn=policy_arn
    )
    print(json.dumps(response, indent=2))


## SageMakerEdgeManagerの設定

### デバイスフリートの作成
* デバイスフリートはデバイスを束ねる概念
* デバイスフリートを作成するとロールエイリアスも自動で作成される。

In [ ]:
# sagemaker_edgemaneger_thing_role_arn の作成反映が間に合わずエラーで落ちることがあるが、再実行でうまくいく

sleep(10)

device_fleet_name ="sagemaker-edge-demo-device-fleet"
role_alias_name = 'SageMakerEdge-' + device_fleet_name
try:
    sagemaker_client.delete_device_fleet(
        DeviceFleetName=device_fleet_name
    )
    print('deleted device freet')
except:
    print('nothing to delete device freet')

try:
    iot_client.delete_role_alias(
        roleAlias=role_alias_name
    )
    print('delete role alias')
except:
    print('nothing to delete role alias')
    
sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=sagemaker_edgemaneger_thing_role_arn,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location
    }
)

## デバイスの登録

In [ ]:
device_name = str(uuid.uuid4()) # Device Name は 36 文字である必要あり。必ずしも UUID である必要はないが（動作確認済） 後述の config ファイルのキーでuuidと書いてあるので、uuidを利用する

try:
    sagemaker_client.deregister_devices(
        DeviceFleetName=device_fleet_name,
        DeviceNames=[device_name],
    )
    print('deregistered device')
except:
    print('nothing to deregister devices')


sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device"
        }
    ]
)

## X.509クライアント証明書の作成と設定

### 証明書の作成

In [ ]:
# 証明書と鍵の作成(IoT の証明書画面に反映される)
iot_cert = iot_client.create_keys_and_certificate(
    setAsActive=True
)
print(iot_cert)
# 証明書を保存
with open('./iot.pem.crt', 'w') as f:
    for line in iot_cert['certificatePem'].split('\n'):
        f.write(line)
        f.write('\n')

# 秘密鍵を保存
with open('./iot_key.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PrivateKey'].split('\n'):
        f.write(line)
        f.write('\n')

# 公開鍵を保存
with open('./iot_key_pair.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PublicKey'].split('\n'):
        f.write(line)
        f.write('\n')

!ls -l ./iot*

### 証明書とロールエイリアスを紐付けるポリシーを証明書にアタッチ
* 証明書を持っていればエッジでロールを引き受けられる（=AWSのサービスを扱える）ようになる

In [ ]:
# ロールエイリアスの arn を取得
role_alias_arn = iot_client.describe_role_alias(
    roleAlias=role_alias_name
)['roleAliasDescription']['roleAliasArn']
print(f'role_alias_arn: {role_alias_arn}')

In [ ]:
# ポリシー作成
alias_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": "iot:AssumeRoleWithCertificate",
    "Resource": role_alias_arn
  }
}
        
certificate_policy_name = 'assume-role-with-certificate-policy'

alias_policy = iot_client.create_policy(
    policyName=certificate_policy_name,
    policyDocument=json.dumps(alias_policy),
)

iot_client.attach_policy(
    policyName=certificate_policy_name,
    target=iot_cert['certificateArn']
)

### IoTエンドポイントを取得
* AWSとエッジがやりとりするためのエンドポイント(URL)を取得

In [ ]:
## IoT エンドポイントを取得
iot_endpoint = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)
endpoint = f'https://{iot_endpoint["endpointAddress"]}/role-aliases/{role_alias_name}/credentials'
print(f'endpoint: {endpoint}')

### RootCAの取得

In [ ]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

### IoTエンドポイントの動作確認
* 作成した証明書と、その証明書の RootCA を使ってエンドポイントにアクセスできるかを確認する
* 問題なければその証明書に紐づくロールエイリアスの credentials が手に入る

In [ ]:
!curl --cert iot.pem.crt --key iot_key.pem.key --cacert AmazonRootCA1.pem $endpoint

### 証明書たちをS3にアップロードする
* SageMaker EdgeManager のエージェントをエッジにインストール後、起動前の設定で使用する

In [ ]:
root_ca_path = sess.upload_data('AmazonRootCA1.pem', bucket, iot_folder)
device_cert_path = sess.upload_data('iot.pem.crt', bucket, iot_folder)
device_key_path = sess.upload_data('iot_key.pem.key', bucket, iot_folder)
print(f'root_ca_path: {root_ca_path}')
print(f'device_cert_path: {device_cert_path}')
print(f'device_key_path: {device_key_path}')

## エッジの準備と推論

### Agentをエッジにダウンロード
ここからエッジデバイスの操作を行う。本来の(EC2ではない)エッジデバイスを操作するためには、コンソールに入るか、SSHでリモートログインして行うことが多いが、今回は EC2 のため、Systems Manager の Session Manager 機能を使って、このノートブックから shell コマンドを送り込む。
* Agent アプリは S3 でホストされているので、エッジ側でそのアプリをダウンロードする
* tar.gz で固めてあるので、展開するコマンドを実行する
* 他設定ファイル(agent.env)の中身を設定する
* bin/以下の実行ファイルについて実行権限を与える（最終的にはインストールディレクトリにコピーされるが、コピー元のファイルに実行権限を先行して与えておく）

In [ ]:
# OSごとのバケットを定義
release_bucket_map = {
    'armv8' : 'sagemaker-edge-release-store-us-west-2-linux-armv8',
    'linux' : 'sagemaker-edge-release-store-us-west-2-linux-x64',
    'win64' : 'sagemaker-edge-release-store-us-west-2-windows-x64',
    'win32' : 'sagemaker-edge-release-store-us-west-2-windows-x86',
}
# EC2 を Ubuntu で立ち上げているため Linux を選択
release_bucket = release_bucket_map['linux']

In [ ]:
# 最新バージョンのプレフィックスを取得
response = s3_client.list_objects(
    Bucket=release_bucket,
    Prefix='Releases/',
)
agent_version_list = []
for content in response['Contents']:
    if content['Key'][-4:] == '.tgz':
        agent_version_list.append(content['Key'])
agent_version_list.sort()
agent_latest_version_prefix = agent_version_list[-1]
print(agent_latest_version_prefix)

In [ ]:
# エッジで実行するコマンドの定義
agent_download_dir = '/sagemaker_agent'
agent_install_dir = '/opt/aws/sagemaker_edge'
# root で実行される
command_str = f"""#!/bin/bash
mkdir -p {agent_download_dir}
mkdir -p {agent_install_dir}
cd {agent_download_dir}
aws s3 cp s3://{release_bucket}/{agent_latest_version_prefix} agent.tgz
tar -xf agent.tgz
sudo sed -i  "s/<unix socket address>/\/tmp\/sagemaker_edge_agent_example.sock/g" agent.env
sudo sed -i "s/<path to agent configuration file>/sagemaker_edge_config.json/g" agent.env
chmod +x {agent_download_dir}/bin/sagemaker_edge_agent_binary
chmod +x {agent_download_dir}/bin/sagemaker_edge_agent_client_example
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

## 各種ファイルをエッジ側でダウンロード
* 証明書や、config.json、パッケージ化したモデルなどをS3からエッジ側にダウンロードする

In [ ]:
certificate_dir = f'{agent_install_dir}/certificates'
iot_credential_dir = f'{agent_install_dir}/iot-credentials'
darknet_model_dir = f'{agent_install_dir}/darknet_model'
keras_model_dir = f'{agent_install_dir}/keras_model'

command_str = f"""#!/bin/bash
cd {agent_install_dir}
mkdir -p {certificate_dir}
aws s3 cp s3://{release_bucket}/Certificates/{region}/{region}.pem {certificate_dir}
chmod -R 444 {certificate_dir}
mkdir -p {iot_credential_dir}
aws s3 cp {root_ca_path} {iot_credential_dir}
aws s3 cp {device_cert_path} {iot_credential_dir}
aws s3 cp {device_key_path} {iot_credential_dir}
chmod -R 444 {iot_credential_dir}
aws s3 cp {keras_img_path} {agent_install_dir}
aws s3 cp {keras_model_data} {agent_install_dir}
mkdir -p {keras_model_dir}
tar -xf {keras_model_package} -C {keras_model_dir}
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

## Agentのconfigファイルの配置とAgentのdaemonを起動
* 1.20210305.a4bc999リリースから、SageMaker EdgeManager の Agent が Daemon 化された
* install.sh を実行すると daemon の登録から自動起動まで動く

In [ ]:
# 設定ファイル作成
capture_prefix = 'demo_capture'
sagemaker_edge_config = json.dumps({
    'sagemaker_edge_core_device_uuid': device_name,
    'sagemaker_edge_core_device_fleet_name': device_fleet_name,
    'sagemaker_edge_core_capture_data_buffer_size': 30,
    'sagemaker_edge_core_capture_data_batch_size': 10,
    'sagemaker_edge_core_capture_data_push_period_seconds': 4,
    'sagemaker_edge_core_folder_prefix': capture_prefix,
    'sagemaker_edge_core_region': region,
    'sagemaker_edge_core_root_certs_path': f'{agent_install_dir}/certificates',
    'sagemaker_edge_provider_aws_ca_cert_file': f'{agent_install_dir}/iot-credentials/AmazonRootCA1.pem',
    'sagemaker_edge_provider_aws_cert_file': f'{agent_install_dir}/iot-credentials/iot.pem.crt',
    'sagemaker_edge_provider_aws_cert_pk_file': f'{agent_install_dir}/iot-credentials/iot_key.pem.key',
    'sagemaker_edge_provider_aws_iot_cred_endpoint': endpoint,
    'sagemaker_edge_provider_provider': 'Aws',
    'sagemaker_edge_provider_s3_bucket_name': bucket,
    'sagemaker_edge_core_capture_data_destination': 'Cloud'
},indent=4)
print(sagemaker_edge_config)

In [ ]:
sagemaker_edge_config_file = 'sagemaker_edge_config.json'
with open(sagemaker_edge_config_file,'w') as f:
    f.write(sagemaker_edge_config)

In [ ]:
config_path = sess.upload_data(sagemaker_edge_config_file, bucket, iot_folder)
print(config_path)

In [ ]:
command_str = f"""#!/bin/bash
aws s3 cp {config_path} {agent_install_dir}
cd {agent_download_dir}
rm -f /tmp/sagemaker_edge_agent_example.sock
echo {agent_install_dir} | bash ./install.sh
# ./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
agent_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
agent_out_id=agent_out['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 3:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=agent_out_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(20)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    pass

### モデルをAgentにロード
* Agent を利用して推論する前に Agent 側にモデルをロードする必要がある。
* LoadModel API を利用する

In [ ]:
keras_model_name = 'demo-keras'
command_str = f"""#!/bin/bash
cd {agent_install_dir}
./bin/sagemaker_edge_agent_client_example LoadModel keras_model {keras_model_name}
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    pass

### ロードしたモデルの確認
* ロードしたモデルを確認するには ListModels API を利用する
* モデルのロードは複数行うことができる(前項の[モデルをAgentにロード](#モデルをAgentにロード)で他のモデルもロードすればよい)ので、現在ロードしているモデル一覧を表示させる

In [ ]:
command_str = f"""#!/bin/bash
cd {agent_install_dir}
./bin/sagemaker_edge_agent_client_example ListModels
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

### Agentを利用して推論
* Agentを利用して推論するには、Predict API を利用する。
* 引数には、①ロードしたモデルの名前②推論したいファイル③入力層④Tensorの形、を指定する

In [ ]:
command_str = f"""#!/bin/bash
cd {agent_install_dir}
./bin/sagemaker_edge_agent_client_example Predict {keras_model_name} keras.bmp input_1 224 224 3
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

In [ ]:
sagemaker_client.describe_device(
    DeviceName=device_name,
    DeviceFleetName=device_fleet_name
)

### データのキャプチャ
* エッジで推論するだけでなく、推論の入力と出力をS3に保存したい場合は PredicAndCapture API を利用する
* 引数は Predict API と同じ
* S3 への保存先は config.json で設定する

In [ ]:
command_str = f"""#!/bin/bash
cd {agent_install_dir}
./bin/sagemaker_edge_agent_client_example PredictAndCapture {keras_model_name} keras.bmp input_1 224 224 3
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

In [ ]:
# Capture データの確認
response = s3_client.list_objects(
    Bucket=bucket,
    Prefix=f'{capture_prefix}/{device_fleet_name}/{keras_model_name}/',
)
key_list = []
input_tensor_list = []
output_tensor_list = []
for content in response['Contents']:
    if content['Key'][-6:] == '.jsonl':
        key_list.append(content['Key'])
    elif 'raw-data/input-tensors' in content['Key']:
        input_tensor_list.append(content['Key'])
    elif 'raw-data/output-tensors' in content['Key']:
        output_tensor_list.append(content['Key'])
key_list.sort(reverse=True)
input_tensor_list.sort(reverse=True)
output_tensor_list.sort(reverse=True)

txt = s3_client.get_object(Bucket=bucket,Key=key_list[0])['Body'].read().decode('utf-8')
print(key_list[0])
print(txt)

# Rawdata はこちら
# input_binary_data = s3_client.get_object(Bucket=bucket,Key=input_tensor_list[0])['Body'].read()
# binary_data = s3_client.get_object(Bucket=bucket,Key=output_tensor_list[0])['Body'].read()

### モデルをAgentからアンロード
* 使わなくなったモデルはアンロードできる。
* UnloadModel API を利用する。引数はモデルの名前のみ

In [ ]:
command_str = f"""#!/bin/bash
cd {agent_install_dir}
./bin/sagemaker_edge_agent_client_example UnloadModel {keras_model_name}
"""
command_list = command_str.split('\n')
print('実行コマンド\n----------')
for command in command_list:
    print(command)

In [ ]:
document_name = "AWS-RunShellScript"
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName=document_name,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands': command_list
    }
)
command_id=response['Command']['CommandId']
print(response)
retry_cnt = 0
while retry_cnt < 20:
    try:
        response = ssm_client.get_command_invocation(
            CommandId=command_id,
            InstanceId=instance_id,
        )
        print(response['Status'])
        if response['Status'] in ['Success','Failed']:
            print(response['StandardOutputContent'])
            break
        else:
            retry_cnt += 1
            sleep(10)
    except:
        print('invocation does not exist yet')
        retry_cnt += 1
        sleep(10)

In [ ]:
# 標準エラー出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stderr'
try:
    txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    print(txt)
except:
    print('No Error')

In [ ]:
# 標準出力を確認
key=f'{folder}/{command_id}/{instance_id}/awsrunShellScript/0.awsrunShellScript/stdout'
txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
print(txt)

## おかたづけ
* デバイス削除解除
* デバイスフリート削除
* ロールエイリアス削除＆デタッチ
* 証明書削除
* IoT モノとタイプを削除
* EC2削除
を行う

In [ ]:
# デバイス登録解除
try:
    sagemaker_client.deregister_devices(
        DeviceFleetName=device_fleet_name,
        DeviceNames=[device_name],
    )
    print('deregistered device')
except:
    print('nothing to deregister devices')

In [ ]:
# デバイスフリート削除
try:
    sagemaker_client.delete_device_fleet(
        DeviceFleetName=device_fleet_name
    )
    print('deleted device freet')
except:
    print('nothing to delete device freet')
# ロールエイリアスの削除
try:
    iot_client.delete_role_alias(
        roleAlias=role_alias_name
    )
    print('delete role alias')
except:
    print('nothing to delete role alias')

In [ ]:
# ロールの削除（存在する場合のみ）
sagemaker_edgemaneger_thing_role_name = 'sagemaker-edgemanager-thing-role'
policy_arn_list = [
    'arn:aws:iam::aws:policy/service-role/AmazonSageMakerEdgeDeviceFleetPolicy',
    'arn:aws:iam::aws:policy/service-role/AWSIoTRuleActions',
    'arn:aws:iam::aws:policy/service-role/AWSIoTLogging',
    'arn:aws:iam::aws:policy/service-role/AWSIoTThingsRegistration'
]

for policy_arn in policy_arn_list:    
    try:
        response = iam_client.detach_role_policy(RoleName=sagemaker_edgemaneger_thing_role_name,PolicyArn=policy_arn)
        print(f'detached {policy_arn}')
        print(json.dumps(response, indent=2))
    except:
        print(f'not attach {policy_arn}')


try:
    response = iam_client.delete_role(RoleName = sagemaker_edgemaneger_thing_role_name)
    print('delete sagemaker edgemanager thing role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete role')


In [ ]:
# X.509 証明書を削除
response = iot_client.update_certificate(
    certificateId=iot_cert['certificateId'],
    newStatus='INACTIVE'
)
print(response)
response = iot_client.detach_policy(
    policyName=certificate_policy_name,
    target=iot_cert['certificateArn']
)
print(response)
response = iot_client.delete_policy(
    policyName=certificate_policy_name
)
print(response)
response = iot_client.delete_certificate(
    certificateId=iot_cert['certificateId']
)
print(response)

In [ ]:
try:
    iot_client.delete_thing(thingName=iot_thing_name)
    print('deleted thing')
except:
    print('nothing to delete thing')
try:
    iot_client.deprecate_thing_type(thingTypeName=iot_thing_type)
    print('deprecated thing type')
    sleep(60*5 + 10) # deprecate から5分立たないと delete できない。10 は buffer
except:
    print('nothing to deprecate thing type')
try:
    iot_client.delete_thing_type(thingTypeName=iot_thing_type)
    print('deleted thing type')
except:
    print('nothing to delete thing type')

In [ ]:
# EC2 の終了
response = ec2_client.terminate_instances(
    InstanceIds=[
        instance_id,
    ]
)
print(response)

In [ ]:
# EC2のロールを削除
# ssm ポリシー
for policy_arn in ec2_role_attach_policy_arn_list:                      
    try:
        response = iam_client.detach_role_policy(RoleName=ec2_role_name,PolicyArn=policy_arn)
        print(f'detach ec2 role {policy_arn}')
        print(json.dumps(response, indent=2))
    except:
        print(f'could not detach {policy_arn}')

policies = iam_client.list_policies()
for policy in policies['Policies']:
    policy_arn = policy['Arn'] if policy['PolicyName'] in ec2_inline_policy_name else None
    if policy_arn:
        # Policy をデタッチ
        try:
            response = iam_client.detach_role_policy(
                RoleName=ec2_role_name,
                PolicyArn=policy_arn
            )
            print(f'detach {ec2_role_name} {policy_arn}')
            print(respoonse)
        except:
            print('did not detach role policy')
        # Policy バージョンの削除
        try:
            for policy_version in iam_client.list_policy_versions(PolicyArn=policy_arn)['Versions']:
                response = iam.delete_policy_version(PolicyArn=policy_arn,VersionId=policy_version['VersionId'])
                print(f'delete policy version {policy_arn} {policy_version["VersionId"]}')
                print(response)
        except:
            print('did not delete policy version')
        # Policy を削除
        try:
            response = iam_client.delete_policy(PolicyArn=policy_arn)
            print(f'delete policy {policy_arn}')
            print(response)
        except:
            print('did not delete policy')

try:
    response = iam_client.detach_role_policy(RoleName=ec2_role_name,PolicyArn=ssm_policy_arn)
    print('detach administrator access')
    print(json.dumps(response, indent=2))
except:
    print('nothing to detach ssm policy')

try:
    response = iam_client.remove_role_from_instance_profile(
        InstanceProfileName=ec2_role_name,
        RoleName=ec2_role_name
    )
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to remove role from instance profile')

try:
    response = iam_client.delete_role(RoleName = ec2_role_name)
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete role')
try:
    response = iam_client.delete_instance_profile(InstanceProfileName=ec2_role_name)
    print('delete instance profile')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete instance profile')